In [8]:
#https://medium.com/@vedantjumle/class-conditioned-diffusion-models-using-keras-and-tensorflow-9997fa6d958c
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env ROCM_PATH=/opt/rocm
import tensorflow as tf
tf.test.is_built_with_rocm()
# document export HSA_OVERRIDE_GFX_VERSION=10.3.0
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

from tensorflow.python.framework.ops import disable_eager_execution,enable_eager_execution
#disable_eager_execution()

env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: ROCM_PATH=/opt/rocm


In [9]:
#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
p =  configParser.get('extractFacesLatents', 'dbChunk')

# TODO Better display of progress and handling of exceptions
contLoop = True # Flag to continue to get chunks of videos from database
con = sl.connect(datasetPathDatabase)
offset = 0
while(contLoop):
    data = con.execute('''SELECT F.LATENT_REP,A.SPEAKER_EMB FROM VIDEO V 
                        INNER JOIN FACE F ON V.ID = F.VIDEO_ID 
                       INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID 
                       WHERE F.LATENT_REP IS NOT NULL AND AUDIO_PRE = 1 AND FACES_PRE = 1
                        ''')
    contLoop = False
    offset = offset + int(p)
    print("Got chunk of DATA from database. Training diffusion network")
    dataGotten = data.fetchall()
    for latent_rep_pickled, speaker_emb_pickled in dataGotten:
        latent_rep = pickle.loads(latent_rep_pickled)
        speaker_emb = pickle.loads(speaker_emb_pickled)


con.close()
        
        





Got chunk of DATA from database. Training diffusion network


In [ ]:
#https://towardsdatascience.com/stable-diffusion-using-hugging-face-501d8dbdd8